### import library

In [1]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm

c:\Users\weightwater\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### prepare data

In [3]:
t = torch.tensor([[1, 2, 3], [4, 5, 6], [4, 6, 7]])
print(t[:-1])

tensor([[1, 2, 3],
        [4, 5, 6]])


In [4]:
def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x
    
    return torch.cat((left, right), dim=0)
    
def concat_feat(x, concat_n):
    assert concat_n % 2 == 1
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n)
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2)
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)
    
    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1402):
    class_num = 41
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
        phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

        for line in phone_file:
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
    if split == 'train' or split == 'val':
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')
    
    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39*concat_nframes)
    if mode != 'test':
        y = torch.empty(max_len, dtype=torch.long)
    
    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
            label = torch.LongTensor(label_dict[fname])
        
        X[idx: idx+cur_len, :] = feat
        if mode != 'test':
            y[idx: idx+cur_len] = label
        
        idx += cur_len
    
    X = X[:idx, :]
    if mode != 'test':
        y = y[:idx]
    
    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
        print(y.shape)
        return X, y
    else:
        return X

#### Define Dataset

In [5]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [6]:
class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = y
        else:
            self.label = None
        
    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]
        
    def __len__(self):
        return len(self.data)

#### Define Model

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.Dropout(0.2),
            nn.LeakyReLU()
        )

    def forward(self, x):
        x = self.block(x)
        return x

In [9]:
class Classifier(nn.Module):
    
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()
        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim[0]),
            *[BasicBlock(hidden_dim[i-1], hidden_dim[i]) for i in range(1, hidden_layers)],
            nn.Linear(hidden_dim[-1], output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

#### Hyper parameters

In [17]:
concat_nframes = 15
train_ratio = 0.8

seed = 914122
batch_size = 2048
num_epoch = 500
learning_rate = 1e-4
model_path = './model/model_0912.ckpt'
weight_decay = 1e-2
gamma = 0.2
step_size = 5

input_dim = 39*concat_nframes
hidden_dim = [2048, 2048, 1024, 512, 512, 256]
hidden_layers = len(hidden_dim)

#### Prepare dataset and model

In [11]:
import gc

train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

train_set = LibriDataset(train_X, train_y)
valid_set = LibriDataset(val_X, val_y)

del train_X, train_y, val_X, val_y
gc.collect()

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 3428


3428it [00:07, 449.59it/s]


[INFO] train set
torch.Size([2117130, 585])
torch.Size([2117130])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:01, 438.81it/s]


[INFO] val set
torch.Size([527028, 585])
torch.Size([527028])


In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda


In [15]:
import numpy as np

def same_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [16]:
same_seed(seed)

model = Classifier(input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

#### Training

In [18]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train()
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, train_pred = torch.max(outputs, 1)
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    writer.add_scalar('Loss/train', train_loss/len(train_loader), epoch)
    writer.add_scalar('Accuracy/train', train_acc/len(train_set), epoch)

    if len(valid_set) > 0:
        model.eval()
        with torch.no_grad():
            for i, batch in tqdm(enumerate(val_loader)):
                features, labels = batch
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)

                loss = criterion(outputs, labels)
                
                _, val_pred = torch.max(outputs, 1)
                val_acc += (val_pred.detach() == labels.detach()).sum().item()
                val_loss += loss.item()

        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(valid_set), val_loss/len(val_loader)
            ))
        
        if best_acc < val_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), model_path)
            print('saving model with acc {:.3f}'.format(val_acc/len(valid_set)))
        writer.add_scalar('Loss/valid', val_loss/len(val_loader), epoch)
        writer.add_scalar('Accuracy/valid', val_acc/len(valid_set), epoch)
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

    scheduler.step()

if len(valid_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')
writer.close()

100%|██████████| 1034/1034 [00:41<00:00, 25.09it/s]
258it [00:05, 50.43it/s]


[001/500] Train Acc: 0.562942 Loss: 1.567216 | Val Acc: 0.644114 loss: 1.199973
saving model with acc 0.644


100%|██████████| 1034/1034 [00:35<00:00, 28.74it/s]
258it [00:05, 49.07it/s]


[002/500] Train Acc: 0.638410 Loss: 1.228888 | Val Acc: 0.662202 loss: 1.133569
saving model with acc 0.662


100%|██████████| 1034/1034 [00:35<00:00, 28.76it/s]
258it [00:05, 49.12it/s]


...

[043/500] Train Acc: 0.765948 Loss: 0.851980 | Val Acc: 0.740185 loss: 0.894739
saving model with acc 0.740


100%|██████████| 1034/1034 [00:38<00:00, 26.63it/s]
258it [00:05, 47.95it/s]


[044/500] Train Acc: 0.766046 Loss: 0.852195 | Val Acc: 0.740114 loss: 0.894761


 36%|███▌      | 369/1034 [00:14<00:25, 26.26it/s]


KeyboardInterrupt: 

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

#### test

In [19]:
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:09, 114.53it/s]

[INFO] test set
torch.Size([646268, 585])


In [28]:
test_loader = DataLoader(LibriDataset(test_X), batch_size=2048)

In [74]:
model.eval()
result = torch.empty((0, 1))
for x in tqdm(test_loader):
    pred = torch.argmax(model(x.to(device)), keepdim=True, axis=1)
    # print(pred)
    result = torch.cat((result, pred.to('cpu')), 0)

100%|██████████| 316/316 [00:04<00:00, 70.23it/s]


In [73]:
result = pd.DataFrame({'class': result.flatten().numpy()})
result['class'] = result['class'].astype(int)

result.to_csv('./result.csv')

In [72]:
result['class'] = result['class'].astype(int)
print(result)


IndexError: too many indices for tensor of dimension 2